In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# get mnist data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Conv layer 1

In [3]:
# input placeholders
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1]) # img 28x28x1 (black & white)
Y = tf.placeholder(tf.float32, [None, 10])

# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
#   Conv     -> (?, 28, 28, 32)
#   Pool     -> (?. 14, 14, 32)
L1_conv = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1_relu = tf.nn.relu(L1_conv)
L1_pool = tf.nn.max_pool(L1_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

print(L1_conv)
print(L1_relu)
print(L1_pool)

Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


# Conv layer 2

In [4]:
# L2 ImgIn shape = (?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#   Conv     -> (?, 28, 28, 32)
#   Pool     -> (?. 14, 14, 32)
L2_conv = tf.nn.conv2d(L1_pool, W2, strides=[1, 1, 1, 1], padding='SAME')
L2_relu = tf.nn.relu(L2_conv)
L2_pool = tf.nn.max_pool(L2_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.reshape(L2_pool, [-1, 7 * 7 * 64])

print(L2_conv)
print(L2_relu)
print(L2_pool)
print(L2)

Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)


# Fully Connected (FC, Dense) layer

In [5]:
# Final FC 7x7x64 input -> 10 outputs
W3 = tf.get_variable('W3', shape=[7*7*64, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b

learning_rate = 0.01

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [12]:
# parameters
training_epochs = 15
batch_size = 100

# initialize
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # train my model
    print('Learning stared. It takes sometime.')
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed_dict = {X: batch_xs, Y: batch_ys}
            c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            avg_cost += c / total_batch
        
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
    print('Learning Finished!')
    
    # Test model and check accuracy
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Learning stared. It takes sometime.
Epoch: 0001 cost = 0.159264182
Epoch: 0002 cost = 0.051702295
Epoch: 0003 cost = 0.040082450
Epoch: 0004 cost = 0.033586440
Epoch: 0005 cost = 0.030055836
Epoch: 0006 cost = 0.026136238
Epoch: 0007 cost = 0.025597716
Epoch: 0008 cost = 0.020566410
Epoch: 0009 cost = 0.021439032
Epoch: 0010 cost = 0.019383771
Epoch: 0011 cost = 0.025384668
Epoch: 0012 cost = 0.015393264
Epoch: 0013 cost = 0.018003366
Epoch: 0014 cost = 0.019481178
Epoch: 0015 cost = 0.016543766
Learning Finished!
Accuracy: 0.9855
